In [1]:
import numpy as np
import sys

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from bart_playground.bcf.bcf import BCF
from bart_playground.params import Tree
from bart_playground import *

In [2]:
proposal_probs = {"grow" : 0.5,
                  "prune" : 0.5}
n_samples = 320
generator = DataGenerator(n_samples=n_samples, n_features=2, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="piecewise_flat")
z_rng = np.random.default_rng(0)
z = z_rng.binomial(1, 0.5, n_samples).astype(bool)
y = y + z * 0.5 - 0.5

X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(X, y, z, random_state=42)
np.set_printoptions(suppress=True)
print(y_train[:12])

[-0.02215361  0.39849321  0.08358079  0.50582743  0.50321077 -0.05778941
  0.48579968  0.3886612  -0.54313394  0.65882761  0.02723698 -0.00511971]


In [3]:
print(X_train[0:5, :])

[[0.37365773 0.09446667]
 [0.68071453 0.39363046]
 [0.43209304 0.62730884]
 [0.96623191 0.59604255]
 [0.64386512 0.82276161]]


In [4]:
bcf = BCF(
    n_mu_trees=100,       # Number of prognostic effect trees
    n_tau_trees=100,       # Number of treatment effect trees
    ndpost=100,          # Posterior samples
    nskip=100,            # Burn-in iterations
    random_state=42
)

bcf.fit(X_train, y_train, z_train)

Iterations: 100%|██████████| 200/200 [00:09<00:00, 20.45it/s]


In [5]:
tree_sp : Tree = bcf.sampler.trace[-1].mu_trees[70]

print(tree_sp)
print(tree_sp.vars)
print(tree_sp.leaf_vals)
# print(bcf.sampler.trace[-1].evaluate(z = np.zeros_like(y_train, dtype = bool), X = X_train)[0:10])

np.testing.assert_allclose(bcf.sampler.trace[-1].evaluate(z_train), bcf.sampler.trace[-1].evaluate(z_train, X_train))

X_1 <= 0.867 (split, n = 240)
	X_1 <= 0.607 (split, n = 206)
		Val: -0.035 (leaf, n = 139)
		Val: -0.016 (leaf, n = 67)
	Val: -0.011 (leaf, n = 34)
[ 1  1 -1 -1 -1 -2 -2 -2]
[        nan         nan -0.01145799 -0.03543598 -0.01615224         nan
         nan         nan]


In [6]:
bcf_result = bcf.predict_components(X_test, z_test)

In [7]:
print(bcf_result[0][0:10])
print(bcf_result[1][0:10])

[ 0.03393351 -0.04673758 -0.39256706 -0.1203358   0.15368494  0.1379183
  0.06406533 -0.1713516   0.12529618 -0.00822858]
[0.2910598  0.27810602 0.27812756 0.28658596 0.24263927 0.24092817
 0.21273204 0.29206197 0.23034141 0.29607209]


#### TLearner class

In [8]:
from sklearn import clone

class TLearner:
    def __init__(self, model_treated, model_control):
        self.model_treated = clone(model_treated)
        self.model_control = clone(model_control)

    def fit(self, X, y, z):
        X_treated = X[z]
        y_treated = y[z]
        X_control = X[~z]
        y_control = y[~z]

        # Fit the models.
        self.model_treated.fit(X_treated, y_treated)
        self.model_control.fit(X_control, y_control)
        # return self

    def predict(self, X, z):
        # Predict outcome: if z is True, use model_treated; else, use model_control.
        preds = np.empty(len(X))
        preds[z] = self.model_treated.predict(X[z])
        preds[~z] = self.model_control.predict(X[~z])
        return preds

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
dt = DecisionTreeRegressor(random_state=42)
lr = LinearRegression()

#### Comparison

In [10]:
tlearner_rf = TLearner(model_treated=rf, model_control=rf)
tlearner_rf.fit(X_train, y_train, z_train)
tlearner_lr = TLearner(model_treated=lr, model_control=lr)
tlearner_lr.fit(X_train, y_train, z_train)
tlearner_dt = TLearner(model_treated=dt, model_control=dt)
tlearner_dt.fit(X_train, y_train, z_train)

In [11]:
models = {"bcf" : bcf, 
          "rf" : tlearner_rf, 
          "lr" : tlearner_lr,
          "dt" : tlearner_dt}
results = {}
for model_name, model in models.items():
    results[model_name] = mean_squared_error(y_test, model.predict(X_test, z_test))
results

{'bcf': 0.0265957411855544,
 'rf': 0.027995018511320614,
 'lr': 0.04275738222331536,
 'dt': 0.04965827443298727}

In [12]:
print(mean_squared_error(bcf_result[2], y_test))

0.0265957411855544


In [13]:
print(bcf_result[1][0:10])
print(bcf_result[2][0:10])

[0.2910598  0.27810602 0.27812756 0.28658596 0.24263927 0.24092817
 0.21273204 0.29206197 0.23034141 0.29607209]
[ 0.36622733  0.18484748 -1.02390533  0.05869356  0.50441698  0.47055727
 -0.13926953 -0.5953437   0.42559462  0.29425686]


In [14]:
naive = np.zeros_like(y_test)
naive[z_test] = np.mean(y_train[z_train])
naive[~z_test] = np.mean(y_train[~z_train])

print(mean_squared_error(naive, y_test))

0.11043741638090805


In [15]:
print(mean_squared_error(np.zeros_like(y_test), y_test))

0.22942363300485438
